# 03 - Inference and Prediction

This notebook demonstrates how to run inference on new images using a trained model.

## What you'll learn:
- How to run predictions using `alt.predict()`
- How to process single images and directories
- How to save predictions and overlays
- How to work with the Predictor class for more control

In [ ]:
import altair as alt
import numpy as np
from pathlib import Path
from PIL import Image

## Basic Prediction

The simplest way to run predictions:

In [ ]:
# Predict on a directory of images
# results = alt.predict("run_abc123", images="path/to/images")

# Predict on a single image
# results = alt.predict("run_abc123", images="path/to/image.png")

# Predict on a list of images
# results = alt.predict("run_abc123", images=["img1.png", "img2.png", "img3.png"])

## Saving Predictions

Save predictions directly during inference:

In [ ]:
# Save predictions to a directory
# results = alt.predict(
#     "run_abc123",
#     images="path/to/images",
#     output_dir="predictions/",  # Saves masks here
# )

## Working with Prediction Results

The `PredictionResults` object provides access to all predictions:

In [ ]:
# Example of working with prediction results
example_code = """
results = alt.predict("run_abc123", images="path/to/images")

# Number of predictions
print(f"Predicted {len(results)} images")

# Iterate over predictions
for pred in results:
    print(f"Image: {pred.image_path}")
    print(f"Mask shape: {pred.mask.shape}")
    print(f"Original size: {pred.original_size}")
    print(f"Unique classes: {np.unique(pred.mask)}")
    print()

# Access by index
first_pred = results[0]
print(f"First prediction mask shape: {first_pred.mask.shape}")
"""
print(example_code)

## Individual Prediction Object

Each `Prediction` object contains:

In [ ]:
prediction_info = """
Prediction attributes:
----------------------
pred.image_path      # Path to the input image
pred.mask            # Predicted segmentation mask (H, W) with class indices
pred.probabilities   # Optional probability maps (C, H, W)
pred.original_size   # Original image size (H, W)
pred.metrics         # Optional per-sample metrics

Prediction methods:
-------------------
pred.get_mask_at_original_size()  # Resize mask to original image size
pred.save(output_dir)              # Save mask as PNG
pred.save_overlay(output_dir)      # Save overlay visualization
pred.save_all(output_dir)          # Save mask, overlay, and optionally probabilities
pred.visualize()                   # Create matplotlib visualization
"""
print(prediction_info)

## Saving Individual Predictions

In [ ]:
save_code = """
for pred in results:
    # Save just the mask
    mask_path = pred.save("outputs/masks/")
    print(f"Saved mask to: {mask_path}")
    
    # Save overlay (mask on original image)
    overlay_path = pred.save_overlay(
        "outputs/overlays/",
        alpha=0.5,  # Transparency
    )
    print(f"Saved overlay to: {overlay_path}")
    
    # Save everything
    saved_files = pred.save_all(
        "outputs/all/",
        save_mask=True,
        save_overlay=True,
        save_probabilities=True,  # Save probability maps as .npy
    )
    print(f"Saved files: {saved_files}")
"""
print(save_code)

## Batch Operations

In [ ]:
batch_code = """
# Save all predictions at once
saved_files = results.save_all(
    output_dir="outputs/",
    save_mask=True,
    save_overlay=True,
    alpha=0.5,
)

# Export sample visualizations with grid
results.export_samples(
    output_dir="outputs/samples/",
    n_samples=10,
    alpha=0.5,
)

# Get all masks as numpy arrays
masks = results.to_numpy()
print(f"Got {len(masks)} masks")
"""
print(batch_code)

## Using the Predictor Class

For more control, use the `Predictor` class directly:

In [ ]:
predictor_code = """
from altair.engine.predictor import Predictor

# Load run
run = alt.load("run_abc123")

# Create predictor
predictor = Predictor(
    run=run,
    checkpoint=run.best_checkpoint,
    device="cuda",  # or "cpu"
    palette=[[0,0,0], [255,0,0], [0,255,0]],  # Custom colors
)

# Run prediction
results = predictor.predict(
    images="path/to/images",
    batch_size=4,  # Process multiple images at once
)
"""
print(predictor_code)

## Single Image Prediction

For predicting on a single image or numpy array:

In [ ]:
single_pred_code = """
from altair.engine.predictor import Predictor

# Create predictor
predictor = Predictor(run, checkpoint)

# Predict on a file path
pred = predictor.predict_single("image.png")

# Or predict on a numpy array directly
image = np.array(Image.open("image.png").convert("RGB"))
pred = predictor.predict_single(image)

print(f"Mask shape: {pred.mask.shape}")
print(f"Probabilities shape: {pred.probabilities.shape}")
"""
print(single_pred_code)

## Working with Probabilities

Access soft predictions (probabilities) for post-processing:

In [ ]:
prob_code = """
for pred in results:
    if pred.probabilities is not None:
        # For binary: shape is (H, W)
        # For multiclass: shape is (C, H, W)
        probs = pred.probabilities
        
        # Get confidence map (max probability at each pixel)
        if probs.ndim == 3:
            confidence = probs.max(axis=0)
        else:
            confidence = np.maximum(probs, 1 - probs)
        
        print(f"Mean confidence: {confidence.mean():.3f}")
        print(f"Min confidence: {confidence.min():.3f}")
        
        # Apply custom threshold
        custom_mask = (probs > 0.7).astype(np.uint8)
"""
print(prob_code)

## Visualization

In [ ]:
viz_code = """
for pred in results:
    # Create visualization with matplotlib
    pred.visualize(
        ground_truth=None,  # Optional GT for comparison
        palette=[[0,0,0], [255,0,0]],  # Colors for classes
        alpha=0.5,
        show_error_map=False,
        save_path=f"viz_{pred.image_path.stem}.png",
    )
"""
print(viz_code)

## Resizing Masks to Original Size

If your model uses a fixed input size, masks are predicted at that size. Use `get_mask_at_original_size()` to resize:

In [ ]:
resize_code = """
for pred in results:
    # Mask at model's input size
    print(f"Predicted mask shape: {pred.mask.shape}")
    print(f"Original image size: {pred.original_size}")
    
    # Resize to original size
    full_size_mask = pred.get_mask_at_original_size()
    print(f"Full size mask shape: {full_size_mask.shape}")
    
    # Save at original size
    Image.fromarray(full_size_mask.astype(np.uint8)).save("mask_full_size.png")
"""
print(resize_code)

## Complete Example

In [ ]:
full_example = """
import altair as alt
import numpy as np
from PIL import Image
from pathlib import Path

# 1. Run predictions
results = alt.predict(
    "my_run_id",
    images="test_images/",
    batch_size=4,
)

print(f"Processed {len(results)} images")

# 2. Save all outputs
results.save_all(
    output_dir="predictions/",
    save_mask=True,
    save_overlay=True,
    alpha=0.5,
)

# 3. Process individual predictions
for pred in results:
    # Get mask at original size
    mask = pred.get_mask_at_original_size()
    
    # Calculate statistics
    total_pixels = mask.size
    positive_pixels = (mask > 0).sum()
    coverage = positive_pixels / total_pixels * 100
    
    print(f"{pred.image_path.name}: {coverage:.1f}% coverage")

# 4. Export sample grid
results.export_samples(
    output_dir="predictions/samples/",
    n_samples=10,
)

print("Done!")
"""
print(full_example)

## Next Steps

- **04_export.ipynb**: Export your model for deployment (ONNX, TorchScript)
- **05_custom_config.ipynb**: Advanced configuration options